# Install

# Import

In [1]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
import re,os
import logging
logging.getLogger('pdfminer.pdfinterp').setLevel(logging.ERROR)
logging.getLogger('pdfminer.layout').setLevel(logging.ERROR)
from modules.schedule import ITR1_SECTIONS

# fn

In [2]:
from modules.extract_logic import clean_row,extract_sections,apply_dynamic_headers,is_empty_row_specific
from typing import List, Dict, Any

# Code

In [3]:
# input_file_path = 'OUTPUT/ ITR 1-2 AY 2025-26.pdf'
# output_file_path = input_file_path.replace(".pdf", "_extracted.txt")
# # excel_file_path = "output.xlsx"
# config_path = "config/ITR1_header.json"
# itr1 = ITR1_SECTIONS(input_file_path, output_file_path, config_path)
# itr1

In [4]:
pdf_directory = 'OUTPUT' 
config_path = "config/ITR1_header.json" 
all_itr1_sections = {}
all_pdfs = [f for f in os.listdir(pdf_directory) if f.lower().endswith('.pdf')]
print(len(all_pdfs))
print(all_pdfs)

6
[' ITR 1-2 AY 2024-25.pdf', ' ITR1.pdf', ' ITR 1-2 AY 2023-24.pdf', ' ITR 1-2 AY 2025-26.pdf', ' ITR 1 AY 2023-24.pdf', ' ITR 1 AY 2024-25.pdf']


In [5]:
for idx, pdf_filename in enumerate(all_pdfs):
    input_file_path = os.path.join(pdf_directory, pdf_filename)
    output_file_path = input_file_path.replace(".pdf", "_extracted.txt")
    print(f"Processing {pdf_filename}...")
    try:
        itr1_instance = ITR1_SECTIONS(input_file_path, output_file_path, config_path)
        if not itr1_instance.acknowledgement:
            itr1_instance.acknowledgement = pdf_filename
        all_itr1_sections[itr1_instance.acknowledgement] = itr1_instance
        
    except Exception as e:
        print(f"Error processing {pdf_filename}: {e}")
print("\nProcessing complete.")
print(f"Successfully processed {len(all_itr1_sections)} out of {len(all_pdfs)} files.")

Processing  ITR 1-2 AY 2024-25.pdf...
Processing  ITR1.pdf...
Processing  ITR 1-2 AY 2023-24.pdf...
Processing  ITR 1-2 AY 2025-26.pdf...
Processing  ITR 1 AY 2023-24.pdf...
Processing  ITR 1 AY 2024-25.pdf...

Processing complete.
Successfully processed 6 out of 6 files.


In [6]:
for key in all_itr1_sections.keys():
    print(key)
    print(all_itr1_sections[key].sections.keys())
    print(all_itr1_sections[key].pan_number)
    print("#---#")

772482530180724
dict_keys(['EXEMPT_INCOME', 'PART_E', 'SCHEDULE_80D', 'SCHEDULE_80G_A', 'SCHEDULE_80G_B', 'SCHEDULE_80G_C', 'SCHEDULE_80G_D', 'SCHEDULE_80GGA', 'SCHEDULE_IT', 'SCHEDULE_TDS1', 'SCHEDULE_TDS2', 'SCHEDULE_TDS3', 'SCHEDULE_TCS'])
ADFPJ2996Q
#---#
170091790250523
dict_keys(['EXEMPT_INCOME', 'PART_E', 'SCHEDULE_80D', 'SCHEDULE_80G_A', 'SCHEDULE_80G_B', 'SCHEDULE_80G_C', 'SCHEDULE_80G_D', 'SCHEDULE_80GGA', 'SCHEDULE_IT', 'SCHEDULE_TDS1', 'SCHEDULE_TDS2', 'SCHEDULE_TDS3', 'SCHEDULE_TCS'])
EGCPG0875B
#---#
249294310170623
dict_keys(['EXEMPT_INCOME', 'PART_E', 'SCHEDULE_80D', 'SCHEDULE_80G_A', 'SCHEDULE_80G_B', 'SCHEDULE_80G_C', 'SCHEDULE_80G_D', 'SCHEDULE_80GGA', 'SCHEDULE_IT', 'SCHEDULE_TDS1', 'SCHEDULE_TDS2', 'SCHEDULE_TDS3', 'SCHEDULE_TCS'])
ADFPJ2996Q
#---#
560199660070925
dict_keys(['EXEMPT_INCOME', 'LTCG112A', 'PART_E', 'ScheduleEA10_13A', 'SCHEDULE_80D', 'SECTION_80U', 'SECTION_80DD', 'SCHEDULE_80G_A', 'SCHEDULE_80G_B', 'SCHEDULE_80G_C', 'SCHEDULE_80G_D', 'SCHEDULE_80GGA

In [13]:
metadata_list: List[Dict[str, Any]] = []

for key, itr1_obj in all_itr1_sections.items():
    metadata_list.append({
        'key': key,
        'pan_number': itr1_obj.pan_number,
        'dof_str': itr1_obj.dof,
        'itr1_object': itr1_obj 
    })
metadata_df = pd.DataFrame(metadata_list)
metadata_df

,key,pan_number,dof_str,itr1_object
0,772482530180724,ADFPJ2996Q,18-Jul-2024,<modules.schedule.ITR1_SECTIONS object at 0x7a...
1,170091790250523,EGCPG0875B,25-May-2023,<modules.schedule.ITR1_SECTIONS object at 0x7a...
2,249294310170623,ADFPJ2996Q,17-Jun-2023,<modules.schedule.ITR1_SECTIONS object at 0x7a...
3,560199660070925,ADFPJ2996Q,07-Sep-2025,<modules.schedule.ITR1_SECTIONS object at 0x7a...
4,255890040180623,EWDPM0323M,18-Jun-2023,<modules.schedule.ITR1_SECTIONS object at 0x7a...
5,589177680050724,EWDPM0323M,05-Jul-2024,<modules.schedule.ITR1_SECTIONS object at 0x7a...


In [8]:
itr1=all_itr1_sections["255890040180623"]
itr2=all_itr1_sections["589177680050724"]

In [9]:
df_EXEMPT_INCOME= itr1.get_section("EXEMPT_INCOME")
display(df_EXEMPT_INCOME)

,0,1,2,3
0,255890040180623,18-Jun-2023,EWDPM0323M,NaN
1,RowNumber,Nature_of_Income,Description,Amount
2,(1),(2),(3),(4)
3,Total,None,None,0


In [10]:
df_EXEMPT_INCOME= itr2.get_section("EXEMPT_INCOME")
display(df_EXEMPT_INCOME)

,0,1,2,3
0,589177680050724,05-Jul-2024,EWDPM0323M,NaN
1,RowNumber,Nature_of_Income,Description,Amount
2,(1),(2),(3),(4)
3,Total,None,None,0


In [11]:
assert False

AssertionError: 

In [ ]:
# itr1.export_to_excel(excel_file_path)

In [ ]:
# itr1.start_pattern

In [ ]:
# itr1.end_pattern

In [ ]:
# itr1.extracted_data[33:34]

In [ ]:
df_temp = pd.DataFrame(itr1.extracted_data[147:188])
# df_temp = clean_row(df_temp)
# df_temp = apply_dynamic_headers(df_temp, itr1.config,"ScheduleEA10_13A")
# df_temp

In [ ]:
# cols="SCHEDULE_80D"
# print(itr1.start_pattern.get(cols,"<No found>"))
# print(itr1.end_pattern.get(cols,"<No found>"))
# print(itr1.hdr_row_map.get(cols,"<No found>"))

In [ ]:
# extract_sections(itr1.extracted_data[147:188], itr1.start_pattern,itr1.end_pattern,itr1.hdr_row_map)

In [ ]:
cols="SCHEDULE_80D"
itr1.sections.get(cols,"<No found>")

In [ ]:
df_temp = itr1.get_section(cols)
# display(df_temp)

In [ ]:
# df_temp.to_excel("del.xlsx",index=False)

In [ ]:
# assert False

In [ ]:
itr1.acknowledgement,itr1.dof,itr1.pan_number

In [ ]:
df_EXEMPT_INCOME= itr1.get_section("EXEMPT_INCOME")
display(df_EXEMPT_INCOME)

In [ ]:
df_LTCG112A= itr1.get_section("LTCG112A")
display(df_LTCG112A)

In [ ]:
# df_PART_C= itr1.get_section("PART_C")
# display(df_PART_C)

In [ ]:
# df_PART_D= itr1.get_section("PART_D")
# display(df_PART_D)

In [ ]:
df_PART_E= itr1.get_section("PART_E")
display(df_PART_E)

In [ ]:
df_EA10_13A= itr1.get_section("ScheduleEA10_13A")
display(df_EA10_13A)

In [ ]:
df_80u = itr1.get_section("SECTION_80U")
display(df_80u)

In [ ]:
df_80dd = itr1.get_section("SECTION_80DD")
display(df_80dd)

In [ ]:
df_80G_A= itr1.get_section("SCHEDULE_80G_A")
display(df_80G_A)

In [ ]:
df_80G_B= itr1.get_section("SCHEDULE_80G_B")
display(df_80G_B)

In [ ]:
df_80G_C= itr1.get_section("SCHEDULE_80G_C")
display(df_80G_C)

In [ ]:
df_80G_D= itr1.get_section("SCHEDULE_80G_D")
display(df_80G_D)

In [ ]:
df_80GGA= itr1.get_section("SCHEDULE_80GGA")
display(df_80GGA)

In [ ]:
df_80GGC= itr1.get_section("SCHEDULE_80GGC")
display(df_80GGC)

In [ ]:
df_it= itr1.get_section("SCHEDULE_IT")
display(df_it)

In [ ]:
df_TDS1= itr1.get_section("SCHEDULE_TDS1")
display(df_TDS1)

In [ ]:
df_TDS2= itr1.get_section("SCHEDULE_TDS2")
display(df_TDS2)

In [ ]:
df_TDS3= itr1.get_section("SCHEDULE_TDS3")
display(df_TDS3)

In [ ]:
df_TCS= itr1.get_section("SCHEDULE_TCS")
display(df_TCS)